
# AI Tutor for Student Success

This project is an AI tutor created to support newly admitted students as they begin their academic journey.  
It uses a Large Language Model (LLM) to understand students’ interests and provide personalized guidance and resources.  
As students progress, the tutor adapts by celebrating their successes and offering helpful tips when they struggle.  
The goal is to motivate students, reduce confusion, and help them feel confident and supported in school.

In [2]:
import os
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [10]:
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [11]:
SYSTEM_PROMPT = (
    "You are a friendly tutor for newly admitted students. "
    "Be very simple and not overwhelming.\n"
    "For the given category, output exactly:\n"
    "Action: <one clear action step>\n"
    "Resource: <one helpful resource suggestion>\n"
    "Keep it short."
)

In [13]:
# --- Helper: ask OpenAI for one category ---
def tutor_category_advice(category_name, category_value, profile_summary):
    user_prompt = (
        f"Student summary (for context): {profile_summary}\n\n"
        f"Category: {category_name}\n"
        f"Student input: {category_value}\n\n"
        f"Give exactly 1 Action and 1 Resource."
    )

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2,
    )
    return response.choices[0].message.content.strip()


In [14]:
# --- Collect profile from  (terminal input) ---
def collect_list(prompt_text):
    items = []
    print(prompt_text + " (type 'done' to finish)")
    while True:
        userInput = input("> ").strip()
        if userInput.lower() == "done":
            break
        if userInput:
            items.append(userInput)
    return items

In [15]:
def collect_profile():
    interests = collect_list("Enter interests (ex: CS, nursing, business)")
    goals = collect_list("Enter goals (ex: transfer, internship, GPA target)")

    level = input("Enter starting level (beginner/intermediate): ").strip()

    # constraints (simple)
    time_per_week = input("Time available per week (hours): ").strip()
    language = input("Preferred language (ex: English, French): ").strip()
    job_hours = input("Job hours per week (0 if none): ").strip()

    learning_style = collect_list("Enter preferred learning style(s) (ex: videos, reading, practice)")

    profile = {
        "interests": interests,
        "goals": goals,
        "level": level,
        "constraints": {
            "time_per_week": time_per_week,
            "language": language,
            "job_hours": job_hours,
        },
        "learning_style": learning_style,
    }
    return profile

In [16]:
# --- Main: iterate through the 5 categories and build final response ---
def tutor_model(profile):
    # small summary used as context (keeps the AI consistent)
    summary = {
        "interests": profile["interests"],
        "goals": profile["goals"],
        "level": profile["level"],
        "constraints": profile["constraints"],
        "learning_style": profile["learning_style"],
    }

    categories = [
        ("Interests", profile["interests"]),
        ("Goals", profile["goals"]),
        ("Level", profile["level"]),
        ("Constraints", profile["constraints"]),
        ("Learning Style", profile["learning_style"]),
    ]

    sections = []
    for name, value in categories:
        advice = tutor_category_advice(name, value, summary)
        sections.append(f"### {name}\n{advice}")

    return "\n\n".join(sections)

In [ ]:
# --- Run ---
profile = {
    "interests": ["Computer Science", "Web Development"],
    "goals": ["Transfer to a 4-year college", "Get an internship"],
    "level": "beginner",
    "constraints": {
        "time_per_week": 8,
        "language": "English",
        "job_hours": 20,
    },
    "learning_style": ["practice", "videos"],
}
print("\n--- Tutor Advice ---\n")
display(Markdown(tutor_model(profile)))


--- Tutor Output ---



### Interests
Action: Start a simple web development project using HTML, CSS, and JavaScript.  
Resource: Check out freeCodeCamp's web development curriculum for guided practice.

### Goals
Action: Research transfer requirements for your target 4-year colleges.  
Resource: Visit the website of your desired colleges for their transfer guidelines.

### Level
Action: Start a beginner-friendly online course in web development.  
Resource: Check out freeCodeCamp for interactive coding lessons.

### Constraints
Action: Create a weekly schedule to allocate your 8 hours for studying and practicing web development.  
Resource: Check out Codecademy's free courses for beginner web development tutorials.

### Learning Style
Action: Watch beginner web development tutorials on YouTube and practice coding along.  
Resource: Codecademy (offers interactive coding exercises and video lessons).